<a href="https://colab.research.google.com/github/csharpshooter/CreateImageOverlaysandDepthMask/blob/master/OverlayImageswithDepthMask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
# !pip uninstall pillow

Uninstalling Pillow-7.0.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/PIL/*
    /usr/local/lib/python3.6/dist-packages/Pillow-7.0.0.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled Pillow-7.0.0


In [2]:
# pip install Pillow==6.0.0

     |████████████████████████████████| 2.0MB 2.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [2]:
!git clone https://github.com/csharpshooter/CreateImageOverlaysandDepthMask.git

Cloning into 'CreateImageOverlaysandDepthMask'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 24 (delta 7), reused 22 (delta 5), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [3]:
cd CreateImageOverlaysandDepthMask

/content/CreateImageOverlaysandDepthMask


In [4]:
!git pull

Already up to date.


In [6]:
!mkdir NYU/weights

mkdir: cannot create directory ‘NYU/weights’: File exists


In [7]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./NYU/weights/nyu.h5

--2020-05-05 03:14:08--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.40.163
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.40.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./NYU/weights/nyu.h5’

./NYU/weights/nyu.h 100%[===================>] 164.89M  30.8MB/s    in 6.0s    

2020-05-05 03:14:14 (27.3 MB/s) - ‘./NYU/weights/nyu.h5’ saved [172897376/172897376]



In [8]:
ls

CreateDepthMaskNYU.py  CreateMasks.py  NYU/  OverlayImages.py  temp.png


In [9]:
import os
from random import random
from random import seed

import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

from CreateDepthMaskNYU import CreateDepthMaskNYU
import matplotlib.pyplot as plt
from PIL import Image

Using TensorFlow backend.


In [0]:
model_path = r'./NYU/weights/nyu.h5'

In [0]:
bg_path = r'/content/gdrive/My Drive/MaskRCNNDataset/Background'
fg_path = r'/content/gdrive/My Drive/MaskRCNNDataset/Foreground'
fg_mask_path = r'/content/gdrive/My Drive/MaskRCNNDataset/Foreground/masks'

final_output = r'/content/gdrive/My Drive/MaskRCNNDataset/OverLayedImages'
final_output_mask = r'/content/gdrive/My Drive/MaskRCNNDataset/OverLayedMask'
final_output_dm = r'/content/gdrive/My Drive/MaskRCNNDataset/OverLayedDepthMasks'

In [0]:
fg_image_ext = '.png'
fg_image_flip_ext = '_flip.png'
bg_image_ext = '.jpg'
final_image_ext = '.jpg'
final_output_dm_ext = '.jpg'
batch_images_count = 40000
batch = 1
batch_folder_name = 'batch_'

In [13]:
depth_model = CreateDepthMaskNYU(model_path)

Loading model...

Model loaded (./NYU/weights/nyu.h5).


In [0]:
for x in tqdm(range(0, 1)):
    seed(x)
    init_x = int(np.floor(random() * 79))
    init_y = int(np.floor(random() * 79))
    dict_for_dm = {}

    # if not x == 0 and x % 4 == 0:
    #     batch += 1;

    if not x == 0:
        batch += 1;

    fg_img_path = final_output + os.path.sep + batch_folder_name + str(batch)
    if not os.path.exists(fg_img_path):
        os.mkdir(fg_img_path)

    fg_mask_img_path = final_output_mask + os.path.sep + batch_folder_name + str(batch)
    if not os.path.exists(fg_mask_img_path):
        os.mkdir(fg_mask_img_path)

    fg_dm_img_path = final_output_dm + os.path.sep + batch_folder_name + str(batch)
    if not os.path.exists(fg_dm_img_path):
        os.mkdir(fg_dm_img_path)

    for mx in tqdm(range(1, 101)):

        # if len(dict_for_dm > 0):
        #     depth_model.flush_depth_maps(dict_for_dm)
        #     dict_for_dm = {}

        overlay_file_path = "{0}{1}{2}{3}".format(fg_path, os.path.sep, str(mx), fg_image_ext)
        overlay_file_mask_path = "{0}{1}{2}{3}".format(fg_mask_path, os.path.sep, str(mx), fg_image_ext)

        overlay = cv2.imread(overlay_file_path, cv2.IMREAD_UNCHANGED)
        overlay_mask = cv2.imread(overlay_file_mask_path, cv2.IMREAD_UNCHANGED)

        flip_overlay_path = "{0}{1}{2}{3}".format(fg_path, os.path.sep, str(mx), fg_image_flip_ext)
        flip_overlay_mask_path = "{0}{1}{2}{3}".format(fg_mask_path, os.path.sep, str(mx), fg_image_flip_ext)

        flip_overlay = cv2.imread(flip_overlay_path, cv2.IMREAD_UNCHANGED)
        flip_overlay_mask = cv2.imread(flip_overlay_mask_path, cv2.IMREAD_UNCHANGED)

        for k in (range(1, 101)):
            bg_file_path = "{0}{1}{2}{3}".format(bg_path, os.path.sep, str(k), bg_image_ext)
            background = cv2.imread(bg_file_path, cv2.IMREAD_UNCHANGED)
            empty_image = np.zeros((224, 224, 3), dtype="uint8")

            for i in range(init_x, init_x + overlay.shape[0]):
                for j in range(init_y, init_y + overlay.shape[1]):
                    if x < 20:
                        if overlay[i - init_x, j - init_y, 3] != 0:
                            background[i, j, 0:3] = overlay[i - init_x, j - init_y, 0:3]
                    else:
                        if flip_overlay[i - init_x, j - init_y, 3] != 0:
                            background[i, j, 0:3] = flip_overlay[i - init_x, j - init_y, 0:3]

            for ix in range(0, 3):
                if x < 20:
                    empty_image[init_x:init_x + overlay.shape[0], init_y:init_y + overlay.shape[1], ix] = overlay_mask[
                                                                                                          0:
                                                                                                          overlay.shape[
                                                                                                              0],
                                                                                                          0:
                                                                                                          overlay.shape[
                                                                                                              1]]
                else:
                    empty_image[init_x:init_x + overlay.shape[0], init_y:init_y + overlay.shape[1],
                    ix] = flip_overlay_mask[
                          0:
                          overlay.shape[
                              0],
                          0:
                          overlay.shape[
                              1]]

            final_image_path = fg_img_path + os.path.sep + str(k) + '_p' + str(mx) + '_it' + str(x) + final_image_ext
            cv2.imwrite(final_image_path, background)

            final_image_mask_path = fg_mask_img_path + os.path.sep + str(k) + '_p' + str(mx) + '_it' + str(x) \
                                    + final_image_ext

            empty_image = cv2.cvtColor(empty_image, cv2.COLOR_BGR2GRAY)

            cv2.imwrite(final_image_mask_path, empty_image)

            final_image_dm_path = fg_dm_img_path + os.path.sep + str(k) + '_p' + str(mx) + '_it' + str(
                x) + final_output_dm_ext

            dict_for_dm[final_image_dm_path] = background

            width = background.shape[1] * 2
            height = background.shape[0] * 2
            dim = (width, height)

            # resize image
            resized = cv2.resize(background, dim, interpolation=cv2.INTER_AREA)

            cv2.imwrite('/content/CreateImageOverlaysandDepthMask/temp.png', resized)

            dm = depth_model.get_depth_map(resized, '/content/CreateImageOverlaysandDepthMask/temp.png', 1)

            depth_model.save_dm(dm, final_image_dm_path)

  0%|          | 0/100 [00:00<?, ?it/s]/content/CreateImageOverlaysandDepthMask/CreateDepthMaskNYU.py:54: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(2.24, 2.24))

 80%|████████  | 80/100 [35:57<08:35, 25.77s/it]

In [0]:
drive.flush_and_unmount()